In [14]:

# Imports standards
import pandas as pd
import numpy as np
import pickle
import os
import warnings
from datetime import datetime
import time

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Machine Learning - Modèles
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Machine Learning - Optimisation
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import randint, uniform

# Machine Learning - Évaluation
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    auc,
    ConfusionMatrixDisplay
)

# Configuration
warnings.filterwarnings('ignore')
np.random.seed(42)

# Configuration des graphiques
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

In [15]:
# ============================================================
# MLFLOW INTEGRATION FUNCTIONS (FIXED)
# ============================================================

def setup_mlflow(experiment_name="Churn_Prediction_Pipeline", tracking_uri=None):
    """
    Configure MLflow experiment and tracking
    """
    print(f"{'='*80}")
    print("🚀 CONFIGURATION MLFLOW")
    print(f"{'='*80}\n")
    
    # Set tracking URI - IMPORTANT: Must be absolute path or URI
    if tracking_uri:
        mlflow.set_tracking_uri(tracking_uri)
        print(f"📍 Tracking URI: {tracking_uri}")
    else:
        # Use absolute path for local tracking
        mlruns_path = os.path.abspath("./mlruns")
        mlflow.set_tracking_uri(f"file://{mlruns_path}")
        print(f"📍 Tracking URI: file://{mlruns_path}")
    
    # Set or create experiment
    try:
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            experiment_id = mlflow.create_experiment(
                experiment_name,
                tags={
                    "project": "Customer Churn Prediction",
                    "team": "Data Science",
                    "created_at": datetime.now().isoformat()
                }
            )
            experiment = mlflow.get_experiment(experiment_id)
            print(f"✅ Nouvelle expérience créée: {experiment_name}")
        else:
            mlflow.set_experiment(experiment_name)
            print(f"✅ Expérience existante: {experiment_name}")
        
        print(f"   • Experiment ID: {experiment.experiment_id}")
        print(f"   • Artifact Location: {experiment.artifact_location}")
        
    except Exception as e:
        print(f"⚠️  Erreur lors de la configuration: {str(e)}")
        print(f"   Création d'une nouvelle expérience...")
        experiment_id = mlflow.create_experiment(experiment_name)
        experiment = mlflow.get_experiment(experiment_id)
        mlflow.set_experiment(experiment_name)
    
    print(f"\n{'='*80}")
    print("📌 POUR VISUALISER LES RÉSULTATS:")
    print(f"   Exécutez: mlflow ui --backend-store-uri {mlflow.get_tracking_uri()}")
    print(f"   Puis ouvrez: http://localhost:5000")
    print(f"{'='*80}\n")
    
    return experiment


In [16]:
# ============================================================
# CONFIGURATION
# ============================================================

# Définir le chemin vers les processeurs
PROCESSOR_DIR = 'processors'

# Setup MLflow
mlflow_experiment = setup_mlflow()

🚀 CONFIGURATION MLFLOW

📍 Tracking URI: file://c:\Users\AkramT\Desktop\3eme_s1\projet_MLOps\notebooks\mlruns
⚠️  Erreur lors de la configuration: file://c:\Users\AkramT\Desktop\3eme_s1\projet_MLOps\notebooks\mlruns is not a valid remote uri. For remote access on windows, please consider using a different scheme such as SMB (e.g. smb://<hostname>/<path>).
   Création d'une nouvelle expérience...


MlflowException: file://c:\Users\AkramT\Desktop\3eme_s1\projet_MLOps\notebooks\mlruns is not a valid remote uri. For remote access on windows, please consider using a different scheme such as SMB (e.g. smb://<hostname>/<path>).

In [ ]:
# ============================================================
# 1. CHARGEMENT DES DONNÉES
# ============================================================

def load_preprocessed_data(processor_dir=PROCESSOR_DIR):
    """
    Charge les données preprocessées et les processeurs
    """
    print(f"{'='*80}")
    print("CHARGEMENT DES DONNÉES PREPROCESSÉES")
    print(f"{'='*80}")
    
    # Charger les données
    data_path = os.path.join(processor_dir, 'preprocessed_data.pkl')
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
    
    X_train = data['X_train']
    X_test = data['X_test']
    y_train = data['y_train']
    y_test = data['y_test']
    
    print(f"✅ Données chargées:")
    print(f"   • X_train: {X_train.shape}")
    print(f"   • X_test:  {X_test.shape}")
    print(f"   • y_train: {y_train.shape}")
    print(f"   • y_test:  {y_test.shape}")
    
    # Charger les processeurs
    processors = {}
    processor_files = ['scaler.pkl', 'label_encoders.pkl', 'feature_names.pkl', 'smote_config.pkl']
    
    print(f"\n✅ Processeurs chargés:")
    for filename in processor_files:
        filepath = os.path.join(processor_dir, filename)
        if os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                processors[filename.replace('.pkl', '')] = pickle.load(f)
            print(f"   • {filename}")
    
    # Statistiques sur les classes
    print(f"\n📊 Distribution des classes:")
    print(f"   • Train - Churn: {y_train.sum():,} ({y_train.mean()*100:.2f}%)")
    print(f"   • Train - Normal: {(~y_train.astype(bool)).sum():,} ({(1-y_train.mean())*100:.2f}%)")
    print(f"   • Test  - Churn: {y_test.sum():,} ({y_test.mean()*100:.2f}%)")
    print(f"   • Test  - Normal: {(~y_test.astype(bool)).sum():,} ({(1-y_test.mean())*100:.2f}%)")
    
    return X_train, X_test, y_train, y_test, processors


# Charger les données
X_train, X_test, y_train, y_test, processors = load_preprocessed_data()

CHARGEMENT DES DONNÉES PREPROCESSÉES
✅ Données chargées:
   • X_train: (42070, 40)
   • X_test:  (6000, 40)
   • y_train: (42070,)
   • y_test:  (6000,)

✅ Processeurs chargés:
   • scaler.pkl
   • label_encoders.pkl
   • feature_names.pkl
   • smote_config.pkl

📊 Distribution des classes:
   • Train - Churn: 21,035 (50.00%)
   • Train - Normal: 21,035 (50.00%)
   • Test  - Churn: 741 (12.35%)
   • Test  - Normal: 5,259 (87.65%)


In [ ]:
# ============================================================
# 2. INITIALISATION DES MODÈLES BASELINE
# ============================================================

def initialize_baseline_models():
    """
    Initialise les modèles de base
    """
    print(f"{'='*80}")
    print("🚀 INITIALISATION DES MODÈLES BASELINE")
    print(f"{'='*80}\n")
    
    fraud_ratio = y_train.sum() / len(y_train)
    scale_pos = (1 - fraud_ratio) / fraud_ratio
    
    print(f"📊 Ratio de déséquilibre calculé: {scale_pos:.2f}")
    print(f"   → Sera utilisé pour pondérer les classes\n")
    
    baseline_config = {
        'XGBoost': XGBClassifier(
            n_estimators=150,
            max_depth=7,
            learning_rate=0.05,
            subsample=0.85,
            colsample_bytree=0.85,
            min_child_weight=3,
            gamma=0.1,
            scale_pos_weight=scale_pos,
            random_state=42,
            eval_metric='auc',
            use_label_encoder=False,
            tree_method='hist'
        ),
        
        'LightGBM': LGBMClassifier(
            n_estimators=150,
            max_depth=8,
            learning_rate=0.05,
            num_leaves=40,
            min_child_samples=25,
            subsample=0.85,
            colsample_bytree=0.85,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1,
            verbose=-1,
            importance_type='gain'
        ),
        
        'Random Forest': RandomForestClassifier(
            n_estimators=200,
            max_depth=25,
            min_samples_split=15,
            min_samples_leaf=5,
            max_features='sqrt',
            class_weight='balanced_subsample',
            bootstrap=True,
            random_state=42,
            n_jobs=-1,
            warm_start=False
        ),
        
        'CatBoost': CatBoostClassifier(
            iterations=150,
            depth=7,
            learning_rate=0.05,
            l2_leaf_reg=3,
            border_count=128,
            auto_class_weights='Balanced',
            random_state=42,
            verbose=False,
            task_type='CPU',
            bootstrap_type='Bernoulli',
            subsample=0.85
        ),
        
        'Logistic Regression (ElasticNet)': LogisticRegression(
            penalty='elasticnet',
            C=1.0,
            l1_ratio=0.5,
            solver='saga',
            max_iter=1000,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1,
            warm_start=False
        )
    }
    
    print("✅ Modèles configurés:\n")
    for idx, (model_name, model) in enumerate(baseline_config.items(), 1):
        print(f"   {idx}. {model_name}")
        if hasattr(model, 'n_estimators'):
            print(f"      └─ Estimateurs: {model.n_estimators}")
        elif hasattr(model, 'iterations'):
            print(f"      └─ Itérations: {model.iterations}")
    
    print(f"\n{'='*80}\n")
    return baseline_config


baseline_models = initialize_baseline_models()

🚀 INITIALISATION DES MODÈLES BASELINE

📊 Ratio de déséquilibre calculé: 1.00
   → Sera utilisé pour pondérer les classes

✅ Modèles configurés:

   1. XGBoost
      └─ Estimateurs: 150
   2. LightGBM
      └─ Estimateurs: 150
   3. Random Forest
      └─ Estimateurs: 200
   4. CatBoost
   5. Logistic Regression (ElasticNet)




In [ ]:
# ============================================================
# 3. ENTRAÎNEMENT DES MODÈLES BASELINE AVEC MLFLOW 
# ============================================================

def train_baseline_models_with_mlflow(models_dict, X_train, y_train, X_test, y_test):
    """
    Entraîne les modèles baseline avec tracking MLflow
    """
    print(f"{'='*80}")
    print("🎯 ENTRAÎNEMENT BASELINE AVEC MLFLOW")
    print(f"{'='*80}\n")
    
    performance_tracker = {}
    
    for model_idx, (model_name, model_instance) in enumerate(models_dict.items(), 1):
        print(f"┌{'─'*78}┐")
        print(f"│ [{model_idx}/{len(models_dict)}] {model_name:<66} │")
        print(f"└{'─'*78}┘")
        
        with mlflow.start_run(run_name=f"{model_name}_baseline") as run:
            start = time.time()
            
            try:
                # Log des paramètres
                mlflow.log_param("model_type", type(model_instance).__name__)
                mlflow.log_param("model_category", "Baseline")
                mlflow.log_param("training_samples", len(X_train))
                mlflow.log_param("test_samples", len(X_test))
                
                # Log des hyperparamètres
                model_params = model_instance.get_params()
                for param_name, param_value in model_params.items():
                    if param_value is not None and not callable(param_value):
                        try:
                            mlflow.log_param(f"hp_{param_name}", str(param_value)[:250])
                        except:
                            pass
                
                # Entraînement
                print(f"   🔄 Entraînement en cours...")
                model_instance.fit(X_train, y_train)
                train_duration = time.time() - start
                
                # Prédictions
                predictions = model_instance.predict(X_test)
                predictions_proba = model_instance.predict_proba(X_test)[:, 1]
                
                # Calcul des métriques
                metrics = {
                    'model_object': model_instance,
                    'accuracy': accuracy_score(y_test, predictions),
                    'precision': precision_score(y_test, predictions, zero_division=0),
                    'recall': recall_score(y_test, predictions, zero_division=0),
                    'f1_score': f1_score(y_test, predictions, zero_division=0),
                    'roc_auc': roc_auc_score(y_test, predictions_proba),
                    'train_time_seconds': train_duration,
                    'predictions': predictions,
                    'predictions_proba': predictions_proba,
                    'confusion_matrix': confusion_matrix(y_test, predictions),
                    'run_id': run.info.run_id
                }
                
                performance_tracker[model_name] = metrics
                
                # Log des métriques
                mlflow.log_metric("accuracy", metrics['accuracy'])
                mlflow.log_metric("precision", metrics['precision'])
                mlflow.log_metric("recall", metrics['recall'])
                mlflow.log_metric("f1_score", metrics['f1_score'])
                mlflow.log_metric("roc_auc", metrics['roc_auc'])
                mlflow.log_metric("training_time", train_duration)
                
                # Log matrice de confusion
                tn, fp, fn, tp = metrics['confusion_matrix'].ravel()
                mlflow.log_metric("true_negatives", int(tn))
                mlflow.log_metric("false_positives", int(fp))
                mlflow.log_metric("false_negatives", int(fn))
                mlflow.log_metric("true_positives", int(tp))
                
                # Log du modèle avec signature
                try:
                    signature = infer_signature(X_train, predictions_proba)
                    mlflow.sklearn.log_model(
                        model_instance,
                        artifact_path="model",
                        signature=signature
                    )
                    # Register model separately
                    model_uri = f"runs:/{run.info.run_id}/model"
                    mlflow.register_model(model_uri, f"{model_name.replace(' ', '_')}_Baseline")
                except Exception as e:
                    print(f"   ⚠️  Avertissement lors du log du modèle: {str(e)}")
                    mlflow.sklearn.log_model(
                        model_instance,
                        artifact_path="model"
                    )
                
                # Graphique ROC
                fig, ax = plt.subplots(figsize=(8, 6))
                fpr, tpr, _ = roc_curve(y_test, predictions_proba)
                ax.plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {metrics["roc_auc"]:.3f})')
                ax.plot([0, 1], [0, 1], 'k--', linewidth=1)
                ax.set_xlabel('False Positive Rate')
                ax.set_ylabel('True Positive Rate')
                ax.set_title(f'ROC Curve - {model_name}')
                ax.legend()
                ax.grid(alpha=0.3)
                
                roc_path = f"roc_curve_{model_name.replace(' ', '_')}_baseline.png"
                plt.savefig(roc_path, dpi=150, bbox_inches='tight')
                mlflow.log_artifact(roc_path)
                plt.close()
                if os.path.exists(roc_path):
                    os.remove(roc_path)
                
                print(f"   ✓ ROC-AUC: {metrics['roc_auc']:.4f}")
                print(f"   ✓ F1-Score: {metrics['f1_score']:.4f}")
                print(f"   ✓ Temps: {train_duration:.2f}s")
                print(f"   ✓ Run ID: {run.info.run_id}")
                print()
                
                mlflow.set_tag("stage", "baseline")
                mlflow.set_tag("model_name", model_name)
                mlflow.set_tag("status", "success")
                
            except Exception as e:
                print(f"   ❌ Erreur: {str(e)}\n")
                mlflow.log_param("status", "failed")
                mlflow.log_param("error", str(e))
                mlflow.set_tag("status", "failed")
                continue
    
    print(f"{'='*80}")
    print(f"✅ BASELINE TERMINÉ - {len(performance_tracker)}/{len(models_dict)} modèles")
    print(f"{'='*80}\n")
    
    return performance_tracker


baseline_results = train_baseline_models_with_mlflow(
    baseline_models, 
    X_train, 
    y_train, 
    X_test, 
    y_test
)


🎯 ENTRAÎNEMENT BASELINE AVEC MLFLOW

┌──────────────────────────────────────────────────────────────────────────────┐
│ [1/5] XGBoost                                                            │
└──────────────────────────────────────────────────────────────────────────────┘


Successfully registered model 'XGBoost_Baseline'.
Created version '1' of model 'XGBoost_Baseline'.


   ✓ ROC-AUC: 0.9995
   ✓ F1-Score: 0.9640
   ✓ Temps: 2.55s
   ✓ Run ID: 89dc50c1541c4d9e96d1b390cc4c95a3

┌──────────────────────────────────────────────────────────────────────────────┐
│ [2/5] LightGBM                                                           │
└──────────────────────────────────────────────────────────────────────────────┘


Successfully registered model 'LightGBM_Baseline'.
Created version '1' of model 'LightGBM_Baseline'.


   ✓ ROC-AUC: 0.9995
   ✓ F1-Score: 0.9667
   ✓ Temps: 2.00s
   ✓ Run ID: 06f65f885ef54462a91c234ece686132

┌──────────────────────────────────────────────────────────────────────────────┐
│ [3/5] Random Forest                                                      │
└──────────────────────────────────────────────────────────────────────────────┘


...

   ✓ ROC-AUC: 0.9971
   ✓ F1-Score: 0.9174
   ✓ Temps: 6.59s
   ✓ Run ID: 33bfb294c2de48eb9df4ad00eed83b4d

┌──────────────────────────────────────────────────────────────────────────────┐
│ [4/5] CatBoost                                                           │
└──────────────────────────────────────────────────────────────────────────────┘


Successfully registered model 'CatBoost_Baseline'.
Created version '1' of model 'CatBoost_Baseline'.


   ✓ ROC-AUC: 0.9998
   ✓ F1-Score: 0.9715
   ✓ Temps: 4.17s
   ✓ Run ID: c73db1c4178e4d98a5522be9d4cb47cc

┌──────────────────────────────────────────────────────────────────────────────┐
│ [5/5] Logistic Regression (ElasticNet)                                   │
└──────────────────────────────────────────────────────────────────────────────┘


Successfully registered model 'Logistic Regression (ElasticNet)_Baseline'.
Created version '1' of model 'Logistic Regression (ElasticNet)_Baseline'.


   ✓ ROC-AUC: 0.9998
   ✓ F1-Score: 0.9488
   ✓ Temps: 35.18s
   ✓ Run ID: 9ef6019518f24c5c96b6fa2b1bc58823

✅ BASELINE TERMINÉ - 5/5 modèles



In [ ]:
# ============================================================
# 4. DÉFINITION DES HYPERPARAMÈTRES
# ============================================================

def define_hyperparameter_search_space():
    """
    Définit les espaces de recherche pour l'optimisation
    """
    print(f"{'='*80}")
    print("📋 CONFIGURATION DES ESPACES DE RECHERCHE")
    print(f"{'='*80}\n")
    
    search_spaces = {
        'XGBoost': {
            'n_estimators': randint(100, 500),
            'max_depth': randint(3, 12),
            'learning_rate': uniform(0.01, 0.29),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'gamma': uniform(0, 0.5),
            'min_child_weight': randint(1, 10),
            'reg_alpha': uniform(0, 1),
            'reg_lambda': uniform(0, 2)
        },
        
        'LightGBM': {
            'n_estimators': randint(100, 500),
            'max_depth': randint(-1, 15),
            'learning_rate': uniform(0.01, 0.29),
            'num_leaves': randint(15, 150),
            'min_child_samples': randint(10, 60),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'reg_alpha': uniform(0, 1),
            'reg_lambda': uniform(0, 2),
            'min_split_gain': uniform(0, 0.1)
        },
        
        'Random Forest': {
            'n_estimators': randint(100, 500),
            'max_depth': [15, 20, 25, 30, 35, None],
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'max_features': ['sqrt', 'log2', 0.5, 0.7, 0.9],
            'max_samples': uniform(0.7, 0.3),
            'class_weight': ['balanced', 'balanced_subsample']
        },
        
        'CatBoost': {
            'iterations': randint(100, 500),
            'depth': randint(4, 11),
            'learning_rate': uniform(0.01, 0.29),
            'l2_leaf_reg': uniform(1, 9),
            'border_count': [32, 64, 128, 200, 254],
            'bagging_temperature': uniform(0, 1),
            'random_strength': uniform(0, 2)
        },
        
        'Logistic Regression (ElasticNet)': {
            'C': uniform(0.001, 10),
            'l1_ratio': uniform(0, 1),
            'max_iter': randint(500, 2000),
            'tol': uniform(1e-5, 1e-3)
        }
    }
    
    for model_name, params in search_spaces.items():
        param_count = len(params)
        print(f"🎯 {model_name}")
        print(f"   └─ Nombre d'hyperparamètres: {param_count}")
        print()
    
    print(f"{'='*80}\n")
    return search_spaces


hyperparameter_spaces = define_hyperparameter_search_space()

📋 CONFIGURATION DES ESPACES DE RECHERCHE

🎯 XGBoost
   └─ Nombre d'hyperparamètres: 9

🎯 LightGBM
   └─ Nombre d'hyperparamètres: 10

🎯 Random Forest
   └─ Nombre d'hyperparamètres: 7

🎯 CatBoost
   └─ Nombre d'hyperparamètres: 7

🎯 Logistic Regression (ElasticNet)
   └─ Nombre d'hyperparamètres: 4




In [ ]:
# ============================================================
# 5. OPTIMISATION AVEC MLFLOW 
# ============================================================

def optimize_models_with_mlflow(
    base_models, 
    search_spaces, 
    X_train, 
    y_train, 
    n_iterations=25, 
    cv_folds=3,
    scoring_metric='roc_auc'
):
    """
    Optimise les hyperparamètres avec tracking MLflow
    """
    print(f"{'='*80}")
    print("🚀 OPTIMISATION AVEC MLFLOW")
    print(f"{'='*80}\n")
    
    optimized_models = {}
    optimization_results = {}
    
    for idx, (model_name, base_estimator) in enumerate(base_models.items(), 1):
        print(f"╔{'═'*78}╗")
        print(f"║ [{idx}/{len(base_models)}] {model_name:<66} ║")
        print(f"╚{'═'*78}╝")
        
        with mlflow.start_run(run_name=f"{model_name}_optimization") as parent_run:
            model_start = time.time()
            
            try:
                # Log des paramètres
                mlflow.log_param("model_type", type(base_estimator).__name__)
                mlflow.log_param("model_category", "Optimization")
                mlflow.log_param("n_iterations", n_iterations)
                mlflow.log_param("cv_folds", cv_folds)
                mlflow.log_param("scoring_metric", scoring_metric)
                
                # RandomizedSearchCV
                randomized_optimizer = RandomizedSearchCV(
                    estimator=base_estimator,
                    param_distributions=search_spaces[model_name],
                    n_iter=n_iterations,
                    cv=cv_folds,
                    scoring=scoring_metric,
                    n_jobs=-1,
                    random_state=42,
                    verbose=0,
                    return_train_score=True
                )
                
                print(f"   🔍 Optimisation en cours...")
                randomized_optimizer.fit(X_train, y_train)
                
                optimization_time = time.time() - model_start
                
                # Sauvegarder
                optimized_models[model_name] = randomized_optimizer.best_estimator_
                
                optimization_results[model_name] = {
                    'best_params': randomized_optimizer.best_params_,
                    'best_cv_score': randomized_optimizer.best_score_,
                    'optimization_time': optimization_time,
                    'run_id': parent_run.info.run_id
                }
                
                # Log
                mlflow.log_metric("best_cv_score", randomized_optimizer.best_score_)
                mlflow.log_metric("optimization_time", optimization_time)
                
                for param_key, param_val in randomized_optimizer.best_params_.items():
                    try:
                        mlflow.log_param(f"best_{param_key}", str(param_val)[:250])
                    except:
                        pass
                
                # Log du modèle
                try:
                    signature = infer_signature(X_train, randomized_optimizer.predict_proba(X_train)[:, 1])
                    mlflow.sklearn.log_model(
                        randomized_optimizer.best_estimator_,
                        artifact_path="optimized_model",
                        signature=signature
                    )
                    # Register model
                    model_uri = f"runs:/{parent_run.info.run_id}/optimized_model"
                    mlflow.register_model(model_uri, f"{model_name.replace(' ', '_')}_Optimized")
                except Exception as e:
                    print(f"   ⚠️  Avertissement lors du log du modèle: {str(e)}")
                    mlflow.sklearn.log_model(
                        randomized_optimizer.best_estimator_,
                        artifact_path="optimized_model"
                    )
                
                print(f"   ✅ Best CV Score: {randomized_optimizer.best_score_:.4f}")
                print(f"   ✅ Temps: {optimization_time:.2f}s")
                print(f"   ✅ Run ID: {parent_run.info.run_id}")
                
                if model_name in baseline_results:
                    baseline_score = baseline_results[model_name]['roc_auc']
                    improvement = (randomized_optimizer.best_score_ - baseline_score) * 100
                    mlflow.log_metric("improvement_vs_baseline", improvement)
                    print(f"   📈 Amélioration: {improvement:+.2f}%")
                
                print()
                
                mlflow.set_tag("stage", "optimization")
                mlflow.set_tag("model_name", model_name)
                mlflow.set_tag("status", "success")
                
            except Exception as error:
                print(f"   ❌ Erreur: {str(error)}\n")
                mlflow.log_param("status", "failed")
                mlflow.log_param("error", str(error))
                mlflow.set_tag("status", "failed")
                optimized_models[model_name] = base_estimator
                continue
    
    print(f"{'='*80}")
    print(f"✅ OPTIMISATION TERMINÉE")
    print(f"{'='*80}\n")
    
    return optimized_models, optimization_results


optimized_models, tuning_results = optimize_models_with_mlflow(
    base_models=baseline_models,
    search_spaces=hyperparameter_spaces,
    X_train=X_train,
    y_train=y_train,
    n_iterations=25,
    cv_folds=3,
    scoring_metric='roc_auc'
)

🚀 OPTIMISATION AVEC MLFLOW

╔══════════════════════════════════════════════════════════════════════════════╗
║ [1/5] XGBoost                                                            ║
╚══════════════════════════════════════════════════════════════════════════════╝
   🔍 Optimisation en cours...


Successfully registered model 'XGBoost_Optimized'.
Created version '1' of model 'XGBoost_Optimized'.


   ✅ Best CV Score: nan
   ✅ Temps: 76.15s
   ✅ Run ID: 1406d17bb39d46209b484786c723ba46
   📈 Amélioration: +nan%

╔══════════════════════════════════════════════════════════════════════════════╗
║ [2/5] LightGBM                                                           ║
╚══════════════════════════════════════════════════════════════════════════════╝
   🔍 Optimisation en cours...


Successfully registered model 'LightGBM_Optimized'.
Created version '1' of model 'LightGBM_Optimized'.


   ✅ Best CV Score: 0.9999
   ✅ Temps: 70.37s
   ✅ Run ID: 6aed045a5fb147ffbf85c96385dfa6cb
   📈 Amélioration: +0.04%

╔══════════════════════════════════════════════════════════════════════════════╗
║ [3/5] Random Forest                                                      ║
╚══════════════════════════════════════════════════════════════════════════════╝
   🔍 Optimisation en cours...


Successfully registered model 'Random Forest_Optimized'.
Created version '1' of model 'Random Forest_Optimized'.


   ✅ Best CV Score: 0.9997
   ✅ Temps: 800.24s
   ✅ Run ID: a2d98b39527f42979081613e230c0599
   📈 Amélioration: +0.26%

╔══════════════════════════════════════════════════════════════════════════════╗
║ [4/5] CatBoost                                                           ║
╚══════════════════════════════════════════════════════════════════════════════╝
   🔍 Optimisation en cours...
   ❌ Erreur: 
All the 75 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\AkramT\anaconda3\envs\mlops\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\AkramT\anaconda3\envs\mlops\Lib\site-packages\catboost\cor

Successfully registered model 'Logistic Regression (ElasticNet)_Optimized'.
Created version '1' of model 'Logistic Regression (ElasticNet)_Optimized'.


In [ ]:
# ============================================================
# 6. ÉVALUATION DES MODÈLES OPTIMISÉS
# ============================================================

def evaluate_optimized_models_with_mlflow(optimized_models, X_train, y_train, X_test, y_test):
    """
    Évalue les modèles optimisés avec tracking MLflow
    """
    print(f"{'='*80}")
    print("📊 ÉVALUATION AVEC MLFLOW")
    print(f"{'='*80}\n")
    
    optimized_performance = {}
    
    for model_idx, (model_name, optimized_model) in enumerate(optimized_models.items(), 1):
        print(f"╔{'═'*78}╗")
        print(f"║ [{model_idx}/{len(optimized_models)}] {model_name:<58} ║")
        print(f"╚{'═'*78}╝")
        
        with mlflow.start_run(run_name=f"{model_name}_evaluation") as run:
            eval_start = time.time()
            
            try:
                # Prédictions
                test_predictions = optimized_model.predict(X_test)
                test_probabilities = optimized_model.predict_proba(X_test)[:, 1]
                
                # Métriques
                performance_metrics = {
                    'model_instance': optimized_model,
                    'accuracy': accuracy_score(y_test, test_predictions),
                    'precision': precision_score(y_test, test_predictions, zero_division=0),
                    'recall': recall_score(y_test, test_predictions, zero_division=0),
                    'f1_score': f1_score(y_test, test_predictions, zero_division=0),
                    'roc_auc': roc_auc_score(y_test, test_probabilities),
                    'eval_time': time.time() - eval_start,
                    'test_predictions': test_predictions,
                    'test_probabilities': test_probabilities,
                    'confusion_mat': confusion_matrix(y_test, test_predictions),
                    'run_id': run.info.run_id
                }
                
                optimized_performance[model_name] = performance_metrics
                
                # Log
                mlflow.log_param("model_type", type(optimized_model).__name__)
                mlflow.log_param("model_category", "Optimized_Evaluation")
                
                mlflow.log_metric("accuracy", performance_metrics['accuracy'])
                mlflow.log_metric("precision", performance_metrics['precision'])
                mlflow.log_metric("recall", performance_metrics['recall'])
                mlflow.log_metric("f1_score", performance_metrics['f1_score'])
                mlflow.log_metric("roc_auc", performance_metrics['roc_auc'])
                
                tn, fp, fn, tp = performance_metrics['confusion_mat'].ravel()
                mlflow.log_metric("true_negatives", int(tn))
                mlflow.log_metric("false_positives", int(fp))
                mlflow.log_metric("false_negatives", int(fn))
                mlflow.log_metric("true_positives", int(tp))
                
                if model_name in baseline_results:
                    baseline_metrics = baseline_results[model_name]
                    improvement_roc = (performance_metrics['roc_auc'] - baseline_metrics['roc_auc']) * 100
                    improvement_f1 = (performance_metrics['f1_score'] - baseline_metrics['f1_score']) * 100
                    
                    mlflow.log_metric("improvement_roc_auc", improvement_roc)
                    mlflow.log_metric("improvement_f1_score", improvement_f1)
                
                # Graphiques
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                fpr, tpr, _ = roc_curve(y_test, test_probabilities)
                axes[0].plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {performance_metrics["roc_auc"]:.3f})')
                axes[0].plot([0, 1], [0, 1], 'k--')
                axes[0].set_xlabel('False Positive Rate')
                axes[0].set_ylabel('True Positive Rate')
                axes[0].set_title(f'ROC Curve - {model_name}')
                axes[0].legend()
                axes[0].grid(alpha=0.3)
                
                cm_display = ConfusionMatrixDisplay(performance_metrics['confusion_mat'])
                cm_display.plot(ax=axes[1], cmap='Blues')
                axes[1].set_title(f'Confusion Matrix - {model_name}')
                
                plots_path = f"evaluation_{model_name}.png"
                plt.tight_layout()
                plt.savefig(plots_path, dpi=150, bbox_inches='tight')
                mlflow.log_artifact(plots_path)
                plt.close()
                os.remove(plots_path)
                
                print(f"   ✓ ROC-AUC: {performance_metrics['roc_auc']:.4f}")
                print(f"   ✓ F1-Score: {performance_metrics['f1_score']:.4f}")
                print(f"   ✓ Run ID: {run.info.run_id}")
                print()
                
                mlflow.set_tag("stage", "evaluation")
                mlflow.set_tag("model_name", model_name)
                
            except Exception as error:
                print(f"   ❌ Erreur: {str(error)}\n")
                mlflow.log_param("status", "failed")
                continue
    
    print(f"{'='*80}")
    print(f"✅ ÉVALUATION TERMINÉE")
    print(f"{'='*80}\n")
    
    return optimized_performance


tuned_results = evaluate_optimized_models_with_mlflow(
    optimized_models, X_train, y_train, X_test, y_test
)

📊 ÉVALUATION AVEC MLFLOW

╔══════════════════════════════════════════════════════════════════════════════╗
║ [1/5] XGBoost                                                    ║
╚══════════════════════════════════════════════════════════════════════════════╝
   ✓ ROC-AUC: 0.9997
   ✓ F1-Score: 0.9737
   ✓ Run ID: 80ba6f831ddc4b8bbfa029eacb8945cd

╔══════════════════════════════════════════════════════════════════════════════╗
║ [2/5] LightGBM                                                   ║
╚══════════════════════════════════════════════════════════════════════════════╝
   ✓ ROC-AUC: 0.9997
   ✓ F1-Score: 0.9724
   ✓ Run ID: 880d74172cd441ba9e3ab1b8a704f1d6

╔══════════════════════════════════════════════════════════════════════════════╗
║ [3/5] Random Forest                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
   ✓ ROC-AUC: 0.9975
   ✓ F1-Score: 0.9173
   ✓ Run ID: e1c8a7717c09446db67ac601811fcf23

╔══════════

In [ ]:
# ============================================================
# 7. TABLEAU COMPARATIF
# ============================================================

def display_performance_comparison(baseline_results, tuned_results):
    """
    Affiche un tableau comparatif
    """
    print(f"{'='*80}")
    print("📊 TABLEAU COMPARATIF DES PERFORMANCES")
    print(f"{'='*80}\n")
    
    comparison_data = []
    
    for model_name in baseline_results.keys():
        if model_name in tuned_results:
            baseline = baseline_results[model_name]
            tuned = tuned_results[model_name]
            
            comparison_data.append({
                'Modèle': model_name,
                'Baseline ROC-AUC': f"{baseline['roc_auc']:.4f}",
                'Optimisé ROC-AUC': f"{tuned['roc_auc']:.4f}",
                'Δ ROC-AUC': f"{(tuned['roc_auc'] - baseline['roc_auc'])*100:+.2f}%",
                'Baseline F1': f"{baseline['f1_score']:.4f}",
                'Optimisé F1': f"{tuned['f1_score']:.4f}",
                'Δ F1': f"{(tuned['f1_score'] - baseline['f1_score'])*100:+.2f}%"
            })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    print(comparison_df.to_string(index=False))
    print(f"\n{'='*80}")
    
    best_model_name = max(tuned_results, key=lambda x: tuned_results[x]['roc_auc'])
    best_roc_auc = tuned_results[best_model_name]['roc_auc']
    
    print(f"🏆 MEILLEUR MODÈLE: {best_model_name}")
    print(f"   • ROC-AUC: {best_roc_auc:.4f}")
    print(f"   • F1-Score: {tuned_results[best_model_name]['f1_score']:.4f}")
    print(f"   • Recall: {tuned_results[best_model_name]['recall']:.4f}")
    print(f"{'='*80}\n")
    
    return comparison_df


comparison_table = display_performance_comparison(baseline_results, tuned_results)

📊 TABLEAU COMPARATIF DES PERFORMANCES

                          Modèle Baseline ROC-AUC Optimisé ROC-AUC Δ ROC-AUC Baseline F1 Optimisé F1   Δ F1
                         XGBoost           0.9995           0.9997    +0.02%      0.9640      0.9737 +0.96%
                        LightGBM           0.9995           0.9997    +0.02%      0.9667      0.9724 +0.57%
                   Random Forest           0.9971           0.9975    +0.04%      0.9174      0.9173 -0.01%
                        CatBoost           0.9998           0.9998    +0.00%      0.9715      0.9715 +0.00%
Logistic Regression (ElasticNet)           0.9998           0.9999    +0.01%      0.9488      0.9549 +0.61%

🏆 MEILLEUR MODÈLE: Logistic Regression (ElasticNet)
   • ROC-AUC: 0.9999
   • F1-Score: 0.9549
   • Recall: 1.0000



In [ ]:
# ============================================================
# 8. ENSEMBLE LEARNING
# ============================================================

def build_ensemble_models(optimized_models, meta_learner_config=None):
    """
    Construit des modèles d'ensemble
    """
    print(f"{'='*80}")
    print("🏗️  CONSTRUCTION DES MODÈLES D'ENSEMBLE")
    print(f"{'='*80}\n")
    
    base_estimators = [(name, model) for name, model in optimized_models.items()]
    
    print(f"📦 Estimateurs de base ({len(base_estimators)}):")
    for idx, (estimator_name, _) in enumerate(base_estimators, 1):
        print(f"   {idx}. {estimator_name}")
    
    if meta_learner_config is None:
        meta_learner = LogisticRegression(
            penalty='elasticnet',
            C=0.5,
            l1_ratio=0.3,
            solver='saga',
            max_iter=1500,
            random_state=42,
            class_weight='balanced',
            n_jobs=-1
        )
    else:
        meta_learner = meta_learner_config
    
    print(f"\n🧠 Meta-Learner (Stacking):")
    print(f"   • Type: {type(meta_learner).__name__}")
    
    print(f"\n🔧 Construction du Stacking Classifier...")
    stacking_model = StackingClassifier(
        estimators=base_estimators,
        final_estimator=meta_learner,
        cv=5,
        stack_method='predict_proba',
        n_jobs=-1,
        passthrough=False,
        verbose=0
    )
    
    print(f"\n🔧 Construction du Voting Classifier...")
    
    if 'tuned_results' in globals():
        weights = [tuned_results[name]['roc_auc'] for name in optimized_models.keys()]
        print(f"   ✓ Mode: Soft voting avec pondération")
    else:
        weights = None
        print(f"   ✓ Mode: Soft voting sans pondération")
    
    voting_model = VotingClassifier(
        estimators=base_estimators,
        voting='soft',
        weights=weights,
        n_jobs=-1,
        flatten_transform=True,
        verbose=False
    )
    
    ensemble_collection = {
        'Stacking Ensemble': stacking_model,
        'Weighted Voting Ensemble': voting_model
    }
    
    print(f"\n{'='*80}")
    print(f"✅ Modèles d'ensemble construits avec succès!")
    print(f"{'='*80}\n")
    
    return ensemble_collection


ensemble_models = build_ensemble_models(optimized_models)

🏗️  CONSTRUCTION DES MODÈLES D'ENSEMBLE

📦 Estimateurs de base (5):
   1. XGBoost
   2. LightGBM
   3. Random Forest
   4. CatBoost
   5. Logistic Regression (ElasticNet)

🧠 Meta-Learner (Stacking):
   • Type: LogisticRegression

🔧 Construction du Stacking Classifier...

🔧 Construction du Voting Classifier...
   ✓ Mode: Soft voting avec pondération

✅ Modèles d'ensemble construits avec succès!



In [ ]:
# ============================================================
# 9. ENTRAÎNEMENT DES ENSEMBLES
# ============================================================

def train_and_evaluate_ensembles_with_mlflow(ensemble_models, X_train, y_train, X_test, y_test):
    """
    Entraîne et évalue les ensembles avec MLflow
    """
    print(f"{'='*80}")
    print("🚀 ENSEMBLES AVEC MLFLOW")
    print(f"{'='*80}\n")
    
    ensemble_performance = {}
    
    for ensemble_idx, (ensemble_name, ensemble_model) in enumerate(ensemble_models.items(), 1):
        print(f"╔{'═'*78}╗")
        print(f"║ [{ensemble_idx}/{len(ensemble_models)}] {ensemble_name:<58} ║")
        print(f"╚{'═'*78}╝")
        
        with mlflow.start_run(run_name=f"{ensemble_name}") as run:
            train_start = time.time()
            
            try:
                mlflow.log_param("model_type", type(ensemble_model).__name__)
                mlflow.log_param("model_category", "Ensemble")
                mlflow.log_param("n_base_estimators", len(ensemble_model.estimators))
                
                print(f"   🔄 Entraînement...")
                ensemble_model.fit(X_train, y_train)
                training_duration = time.time() - train_start
                
                predictions = ensemble_model.predict(X_test)
                prediction_probas = ensemble_model.predict_proba(X_test)[:, 1]
                
                ensemble_metrics = {
                    'model_object': ensemble_model,
                    'accuracy': accuracy_score(y_test, predictions),
                    'precision': precision_score(y_test, predictions, zero_division=0),
                    'recall': recall_score(y_test, predictions, zero_division=0),
                    'f1_score': f1_score(y_test, predictions, zero_division=0),
                    'roc_auc': roc_auc_score(y_test, prediction_probas),
                    'training_time': training_duration,
                    'predictions': predictions,
                    'prediction_probas': prediction_probas,
                    'confusion_matrix': confusion_matrix(y_test, predictions),
                    'run_id': run.info.run_id
                }
                
                ensemble_performance[ensemble_name] = ensemble_metrics
                
                mlflow.log_metric("accuracy", ensemble_metrics['accuracy'])
                mlflow.log_metric("precision", ensemble_metrics['precision'])
                mlflow.log_metric("recall", ensemble_metrics['recall'])
                mlflow.log_metric("f1_score", ensemble_metrics['f1_score'])
                mlflow.log_metric("roc_auc", ensemble_metrics['roc_auc'])
                mlflow.log_metric("training_time", training_duration)
                
                tn, fp, fn, tp = ensemble_metrics['confusion_matrix'].ravel()
                mlflow.log_metric("true_negatives", int(tn))
                mlflow.log_metric("false_positives", int(fp))
                mlflow.log_metric("false_negatives", int(fn))
                mlflow.log_metric("true_positives", int(tp))
                
                try:
                    signature = infer_signature(X_train, prediction_probas)
                    mlflow.sklearn.log_model(
                        ensemble_model,
                        artifact_path="ensemble_model",
                        signature=signature,
                        registered_model_name=f"{ensemble_name.replace(' ', '_')}"
                    )
                except:
                    mlflow.sklearn.log_model(
                        ensemble_model,
                        artifact_path="ensemble_model",
                        registered_model_name=f"{ensemble_name.replace(' ', '_')}"
                    )
                
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                fpr, tpr, _ = roc_curve(y_test, prediction_probas)
                axes[0].plot(fpr, tpr, linewidth=3, label=f'ROC (AUC = {ensemble_metrics["roc_auc"]:.3f})')
                axes[0].plot([0, 1], [0, 1], 'k--')
                axes[0].set_xlabel('False Positive Rate')
                axes[0].set_ylabel('True Positive Rate')
                axes[0].set_title(f'ROC Curve - {ensemble_name}')
                axes[0].legend()
                axes[0].grid(alpha=0.3)
                
                cm_display = ConfusionMatrixDisplay(ensemble_metrics['confusion_matrix'])
                cm_display.plot(ax=axes[1], cmap='Greens')
                axes[1].set_title(f'Confusion Matrix - {ensemble_name}')
                
                ensemble_plots_path = f"ensemble_{ensemble_name.replace(' ', '_')}.png"
                plt.tight_layout()
                plt.savefig(ensemble_plots_path, dpi=150, bbox_inches='tight')
                mlflow.log_artifact(ensemble_plots_path)
                plt.close()
                os.remove(ensemble_plots_path)
                
                print(f"   ✓ ROC-AUC: {ensemble_metrics['roc_auc']:.4f}")
                print(f"   ✓ F1-Score: {ensemble_metrics['f1_score']:.4f}")
                print(f"   ✓ Temps: {training_duration:.2f}s")
                print(f"   ✓ Run ID: {run.info.run_id}")
                print()
                
                mlflow.set_tag("stage", "ensemble")
                mlflow.set_tag("model_name", ensemble_name)
                
            except Exception as error:
                print(f"   ❌ Erreur: {str(error)}\n")
                mlflow.log_param("status", "failed")
                continue
    
    print(f"{'='*80}")
    print(f"✅ ENSEMBLES TERMINÉS")
    print(f"{'='*80}\n")
    
    return ensemble_performance


ensemble_results = train_and_evaluate_ensembles_with_mlflow(
    ensemble_models, X_train, y_train, X_test, y_test
)

🚀 ENSEMBLES AVEC MLFLOW

╔══════════════════════════════════════════════════════════════════════════════╗
║ [1/2] Stacking Ensemble                                          ║
╚══════════════════════════════════════════════════════════════════════════════╝
   🔄 Entraînement...


Successfully registered model 'Stacking_Ensemble'.
Created version '1' of model 'Stacking_Ensemble'.


   ✓ ROC-AUC: 0.9999
   ✓ F1-Score: 0.9846
   ✓ Temps: 232.89s
   ✓ Run ID: 5584788fcb374dea90885d4486b53536

╔══════════════════════════════════════════════════════════════════════════════╗
║ [2/2] Weighted Voting Ensemble                                   ║
╚══════════════════════════════════════════════════════════════════════════════╝
   🔄 Entraînement...
   ❌ Erreur: The estimator XGBClassifier should be a classifier.

✅ ENSEMBLES TERMINÉS



In [ ]:
# ============================================================
# 10. COMPARAISON COMPLÈTE
# ============================================================

def comprehensive_model_comparison(baseline_results, tuned_results, ensemble_results):
    """
    Compare tous les modèles
    """
    print(f"{'='*80}")
    print("🏆 COMPARAISON COMPLÈTE DE TOUS LES MODÈLES")
    print(f"{'='*80}\n")
    
    all_results = []
    
    for model_name, metrics in baseline_results.items():
        all_results.append({
            'Modèle': f"{model_name} (Baseline)",
            'Type': 'Baseline',
            'ROC-AUC': metrics['roc_auc'],
            'F1-Score': metrics['f1_score'],
            'Recall': metrics['recall'],
            'Precision': metrics['precision']
        })
    
    for model_name, metrics in tuned_results.items():
        all_results.append({
            'Modèle': f"{model_name} (Optimisé)",
            'Type': 'Optimisé',
            'ROC-AUC': metrics['roc_auc'],
            'F1-Score': metrics['f1_score'],
            'Recall': metrics['recall'],
            'Precision': metrics['precision']
        })
    
    for model_name, metrics in ensemble_results.items():
        all_results.append({
            'Modèle': model_name,
            'Type': 'Ensemble',
            'ROC-AUC': metrics['roc_auc'],
            'F1-Score': metrics['f1_score'],
            'Recall': metrics['recall'],
            'Precision': metrics['precision']
        })
    
    comparison_df = pd.DataFrame(all_results)
    comparison_df = comparison_df.sort_values('ROC-AUC', ascending=False)
    
    print("📊 Classement par ROC-AUC:\n")
    print(comparison_df.to_string(index=False))
    
    champion = comparison_df.iloc[0]
    
    print(f"\n{'='*80}")
    print(f"🥇 MODÈLE CHAMPION: {champion['Modèle']}")
    print(f"{'='*80}")
    print(f"   • ROC-AUC:   {champion['ROC-AUC']:.4f}")
    print(f"   • F1-Score:  {champion['F1-Score']:.4f}")
    print(f"   • Recall:    {champion['Recall']:.4f}")
    print(f"   • Precision: {champion['Precision']:.4f}")
    print(f"{'='*80}\n")
    
    return comparison_df, champion


full_comparison_df, best_model_info = comprehensive_model_comparison(
    baseline_results, 
    tuned_results, 
    ensemble_results
)


print("✅ SCRIPT TERMINÉ AVEC SUCCÈS!")
print(f"🎯 Champion: {best_model_info['Modèle']}")
print(f"📊 ROC-AUC: {best_model_info['ROC-AUC']:.4f}")

🏆 COMPARAISON COMPLÈTE DE TOUS LES MODÈLES

📊 Classement par ROC-AUC:

                                     Modèle     Type  ROC-AUC  F1-Score   Recall  Precision
Logistic Regression (ElasticNet) (Optimisé) Optimisé 0.999900  0.954897 1.000000   0.913687
                          Stacking Ensemble Ensemble 0.999896  0.984615 0.993252   0.976127
Logistic Regression (ElasticNet) (Baseline) Baseline 0.999817  0.948784 1.000000   0.902558
                        CatBoost (Baseline) Baseline 0.999789  0.971504 0.989204   0.954427
                        CatBoost (Optimisé) Optimisé 0.999789  0.971504 0.989204   0.954427
                        LightGBM (Optimisé) Optimisé 0.999744  0.972428 0.975709   0.969169
                         XGBoost (Optimisé) Optimisé 0.999704  0.973666 0.973009   0.974324
                        LightGBM (Baseline) Baseline 0.999535  0.966711 0.979757   0.954008
                         XGBoost (Baseline) Baseline 0.999531  0.964048 0.977058   0.951380
         